In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D,concatenate,Concatenate)
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model
import matplotlib.pyplot as plt

import PIL
from PIL import Image, ImageOps
import cv2

from keras.losses import binary_crossentropy, categorical_crossentropy
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score, cohen_kappa_score
from keras.utils import Sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import imgaug as ia
from keras.applications.vgg16 import VGG16


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
df_test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [ ]:
num_train = df_train.shape[0]
df_train.shape

In [ ]:
num_test = df_test.shape[0]
df_train.shape

In [ ]:
train_images =  "../input/aptos2019-blindness-detection/train_images/"
test_images = "../input/aptos2019-blindness-detection/test_images/"

In [ ]:
df_train.head()

In [ ]:
df_test.head()

# Sample Images

In [ ]:
img = load_img(train_images +  df_train['id_code'].iloc[1] +".png")
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
X = df_train['id_code']
y = df_train['diagnosis']


y.hist()

In [ ]:
num_classes = 5

In [ ]:
y = to_categorical(y, num_classes=num_classes)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)



# Transfer Learning with VGG16

In [ ]:
vgg = VGG16()

In [ ]:
vgg.summary()

In [ ]:
vgg_layers_list = vgg.layers

In [ ]:
model = Sequential()

### removing prediction layer

In [ ]:
for i in range(len(vgg_layers_list)-1):
    model.add(vgg_layers_list[i])

### freeze the pretrained layers

In [ ]:
for layers in model.layers:
    layers.trainable = False

In [ ]:
model.add(Dense(num_classes, activation="softmax"))

In [ ]:
num_classes

In [ ]:
model.compile(loss = "categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

In [ ]:
diagnosis = [0,1,2,3,4]

In [ ]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

In [ ]:
df_train['diagnosis'] = df_train['diagnosis'].astype('str')

In [ ]:
def append_ext(fn):
    return fn+".png"

df_train["id_code"]=df_train["id_code"].apply(append_ext)


In [ ]:
df_test["id_code"]=df_test["id_code"].apply(append_ext)

In [ ]:
train_generator=datagen.flow_from_dataframe(
    dataframe=df_train[:2929],
    directory= "../input/aptos2019-blindness-detection/train_images",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(224,224))

In [ ]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=df_test,
directory="../input/aptos2019-blindness-detection/test_images",
x_col="id_code",
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224,224))

In [ ]:
valid_generator=datagen.flow_from_dataframe(
dataframe=df_train[2929:],
directory="../input/aptos2019-blindness-detection/train_images",
x_col="id_code",
y_col="diagnosis",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=["0", "1", "2", "3", "4"],
target_size=(224,224))

In [ ]:
batch_size = 32

In [ ]:
model.summary()

In [ ]:
model.fit(train_generator,
           steps_per_epoch=num_train//batch_size,
           epochs= 10,
           validation_data=valid_generator,
           validation_steps= num_test//batch_size)